# Récupération et nettoyage des données du modèle

In [3]:
import numpy as np

import pandas as pd  


In [4]:
path1 = 'https://raw.githubusercontent.com/GeoffroyENSAE/Cryptomonnaies-et-ecologie/main/Hashrate.csv'
path2 = 'https://raw.githubusercontent.com/GeoffroyENSAE/Cryptomonnaies-et-ecologie/main/cours.csv'

In [5]:
df1 = pd.read_csv(path1,sep=';') 
df2 = pd.read_csv(path2,sep=';')

In [6]:
df1=df1.join(df1['DateTime'].str.split(' ',expand=True).rename(columns={0:'Date',1:'inutile',2:'Hashrate'}))
df1=df1.drop(['DateTime','inutile'],axis=1)
df2=df2.drop(['Ouv.','Plus Haut','Plus Bas','Vol.','Variation %'],axis=1)

In [7]:
df1['Date'] = pd.to_datetime(df1['Date'])
df2['Date'] = pd.to_datetime(df2['Date'])

In [8]:
df2['Dernier']=df2['Dernier'].replace('.','')

In [9]:
def nettoyer1(x):
    return(float(x.replace(',','.')))

def nettoyer2(x):
    return(float((x.replace('.','')).replace(',','.')))

def string(x):
    return(str(x))

def entier(x):
    return(int(x))

In [10]:
df1['Hash Rate']=df1['Hash Rate'].apply(nettoyer1)
df2['Dernier']=df2['Dernier'].apply(nettoyer2)

In [11]:
df=df1.merge(df2)
df=df.assign(BTC=0)

In [12]:
for i in range(575):
    df['BTC'][i]=3600

for i in range(575,2035):
    df['BTC'][i]=1800
    
for i in range(2035,2557):
    df['BTC'][i]=900

<ipython-input-12-5bf19dc8d0d1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['BTC'][i]=3600
<ipython-input-12-5bf19dc8d0d1>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['BTC'][i]=1800
<ipython-input-12-5bf19dc8d0d1>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['BTC'][i]=900


In [13]:
df=df.assign(Seuil=(df['BTC']*df['Dernier'])/(0.1*24)/(df['Hash Rate']*10**(-12)))

In [14]:
df['Seuil']=df['Seuil'].rolling(50).mean()

In [15]:
path = 'https://raw.githubusercontent.com/GeoffroyENSAE/Cryptomonnaies-et-ecologie/main/equipement.csv'

In [16]:
df3 = pd.read_csv(path,sep=',')

In [17]:
df3=df3.drop(['Unnamed: 8','Unnamed: 9','Unnamed: 1','Unnamed: 3'],axis=1)

In [18]:
df3.drop(df3.index[0:13],0,inplace=True)

In [19]:
df3=df3.rename(columns = {'CBECI SHA-256 Mining Equipment List': 'Mineur', 'Unnamed: 2':'Date' 
                        , 'Unnamed: 1': 'Date de sortie','Unnamed: 4': 'Hashing power (Th/s)', 'Unnamed: 5': 'Puissance (W)', 'Unnamed: 6': 'Efficacité (J/Gh)', 'Unnamed: 7': 'Poids (kg)'})

In [20]:
def date(x):
    return('01.'+x)

In [21]:
df3['Date']=df3['Date'].apply(date)

In [22]:
def inverser(x):
    return(x[3:6]+x[0:3]+x[6:10])

In [23]:
df3['Date']=df3['Date'].apply(inverser)

In [24]:
df3['Date'] = pd.to_datetime(df3['Date'])

In [25]:
df=df.merge(df3,how='outer')

In [26]:
df['Efficacité (J/Gh)'] = df['Efficacité (J/Gh)'].fillna(0)

In [27]:
df['Efficacité (J/Gh)']=df['Efficacité (J/Gh)'].astype(float)

In [28]:
for i in range(2609):
    if df['Efficacité (J/Gh)'][i]==0.:
        df['Efficacité (J/Gh)'][i]=-10

<ipython-input-28-2fa8926b2a51>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Efficacité (J/Gh)'][i]=-10
